### **Variantes de los Bloom filters**

#### **Bloomier Filter**  


Los filtros de Bloom son estructuras probabilísticas diseñadas para responder eficientemente a consultas de pertenencia (¿esta clave es posiblemente miembro del conjunto?). Sin embargo, adolecen de una gran limitación: no permiten **asociar valores** a las claves. El **Bloomier filter** surge como extensión que habilita un mapeo estático de *n* claves a sus valores correspondientes, manteniendo prestaciones de espacio y tiempo muy competitivas y evitando colisiones si se diseña adecuadamente.

Para un conjunto de *n* elementos y una tasa de falso positivo deseada $\delta$ ($0 < \delta < 1$), el Bloom filter clásico usa:

- Un arreglo de *m* bits.  
- *k* funciones de hash independientes.  

Se eligen *m* y *k* tal que:
$$
m = \left\lceil -\frac{n \ln \delta}{(\ln 2)^2} \right\rceil,
\quad
k = \left\lceil \frac{m}{n} \ln 2 \right\rceil.
$$

La probabilidad de falso positivo aproximada es:
$$
P_{fp} \approx \left(1 - e^{-k n / m}\right)^k \approx \delta.
$$

**Objetivo del Bloomier filter**  

Sea $K = \{k_1, k_2, \dots, k_n\}$ el conjunto de claves, con una función objetivo  
$$
f: K \to V,  
$$  
que asigna a cada clave $k_i$ un valor $v_i$. El Bloomier filter debe:

1. **Responder en tiempo O(k)** a consultas: $\mathsf{lookup}(k)$.  
2. **No requerir más de O(m)** espacio adicional, idealmente $m = O(n)$.  
3. **Garantizar descarga** (sin colisiones) de cada $v_i$ para $k_i\in K$.  
4. **Opcionalmente**, producir valores arbitrarios (garbage) para claves fuera de *K*.  

Denotemos:

- $n = |K|$  
- $m$ número de *buckets* (celdas) en el arreglo interno $T[\,0..m-1]$.  
- $k$ número de funciones de hash $h_0,\dots,h_{k-1}$.  
- Para cada clave $k_i$, definimos el vector de posiciones  
  $$
  \mathbf{p}_i = (p_{i,0}, \, p_{i,1}, \dots, p_{i,k-1}),  
  \quad
  p_{i,j} = h_j(k_i) \bmod m.
  $$

La idea central es asignar en cada $T[p_{i,j}]$ un valor tal que:

$$
\bigoplus_{j=0}^{k-1} T[p_{i,j}] \;=\; v_i,
$$

donde $\oplus$ representa la operación XOR bit a bit.  

Por tanto, dado el arreglo $T$, la operación **lookup** es:

```python
def lookup(key):
    result = 0
    for j in range(k):
        pos = h_j(key) % m
        result ^= T[pos]
    return result
```
 

El paso más delicado es poblar el arreglo $T$ de modo que la igualdad de XOR se cumpla para **todas** las *n* claves. El método habitual consiste en:

1. **Calcular** para cada clave $k_i$ su vector de posiciones $\mathbf{p}_i$.  
2. **Construir** un grafo de incidencia:  
   - Los **vértices** son las posiciones $0..m-1$.  
   - Cada clave corresponde a una **arista** que conecta los *k* vértices de $\mathbf{p}_i$.  
3. **Resolver** el grafo en orden degenerado:  
   - Mantener un conjunto _degenerado_ de vértices de grado 1 (solo una arista incidente).  
   - Mientras exista un vértice $v$ de grado 1:
     1. Extraer arista $e_i$ incidente a $v$.  
     2. Asignar  
        $$
        T[v] \;=\; v_i \;\oplus\;\bigoplus_{u \in \mathbf{p}_i,\, u \neq v} T[u].
        $$
     3. Eliminar la arista $e_i$ del grafo y actualizar grados.  
4. Si **todas** las aristas se procesan sin ciclos, el grafo es **acíclico** y la construcción concluye.  
5. En caso de **ciclo** (no quedan vértices de grado 1) se elige otra semilla para re-hashear y se vuelve a intentar.

**Complejidad de construcción**  
- Tiempo esperado: $O(n k)$, pues cada arista se elimina una vez y procesamos $k$ posiciones.  
- Fracaso con baja probabilidad; en la práctica pocas semillas son necesarias.


Una vez $T$ está listo, la operación de consulta es **determinista**:

$$
\mathsf{lookup}(k) = \bigoplus_{j=0}^{k-1} T\bigl(h_j(k)\bigr) \bmod m.
$$

No hay riesgo de **falso negativo**: para cada $k_i\in K$, el XOR reconstruye exactamente $v_i$. Para claves ajenas a $K$, el resultado será un "valor basura" no significativo, pero **no es una fal­sa alarma**—es simplemente no válido.

**Tiempo de consulta**: O(k) accesos a memoria + O(k) XORs. Para $k\approx\ln 2 \,(m/n)$, esto equivale a unos pocos accesos secuenciales.

**Métricas de espacio y parámetros**  

**Tamaño del arreglo**  

- Normalmente se elige  
  $$
  m \;\approx\; c\,n
  $$
  con $c\in [1.2,2]$, en función de la tolerancia $\delta$.  
- Comparación: un hash map estándar en Python usa $\approx 8–16$ bytes por entrada (sin contar overhead del objeto). El Bloomier filter puede rondar $c$ bytes por clave (si $T$ almacena valores de 32 bits).

**Número de hash functions**  

- Se escoge según  
  $$
  k = \left\lceil \ln 2 \;\frac{m}{n} \right\rceil.
  $$
- Trade‑off: aumentar $k$ mejora la desacoplamiento en la construcción (más hiper‑aristas), pero incrementa tiempo de consulta y uso de CPU.

**Overhead total**  

- **Espacio**: $m$ celdas de $\ell$ bits, donde $\ell$ es el tamaño del valor (por ejemplo, 32 bits para enteros).  
- **Tiempo construcción**: $O(n k)$.  
- **Tiempo consulta**: $O(k)$.  

**Probabilidad de fallo de construcción**  

Cuando el grafo no es acíclico, la construcción falla. La probabilidad de volver a intentar suele ser muy baja si $m/n \geq 1.23$ y $k\geq 3$.  

**Comparación con alternativas**  

| Estructura                   | Espacio por clave  | Consulta          | Valores asociados | Falsos positivos |
|-------------------------------|--------------------|-------------------|-------------------|------------------|
| Hash map (open addressing)    | ~16–32 bytes       | O(1) promedio     | Sí                | No               |
| Bloom filter clásico       | ~1.44 · log₂(1/δ)·n bits ≈ (0.69 · n ln(1/δ)) bits | O(k) (∼5–10) | No                | Sí               |
| **Bloomier filter**           | $m$ celdas de $\ell$ bits ≈ $c n ·ℓ$ bits   | O(k)             | Sí                | No (para K)      |

- **Ventaja**: los Bloomier filters soportan mapeo clave→valor sin almacenar claves explícitas.  
- **Desventaja**: no pueden **inserciones dinámicas** ni borrados eficientes; son **estáticos**.


**Ejemplo**

Supongamos $n=10^6$ claves, valores de 32 bits, y deseamos $\delta=10^{-6}$.  

1. Cálculo de $m$:  
   $$
   m \approx -\frac{n \ln \delta}{(\ln 2)^2}
     = -\frac{10^6 \ln(10^{-6})}{0.4809}
     = \frac{10^6 \times 13.8155}{0.4809}
     ≈ 2.87\times10^7 \text{ celdas}.
   $$
2. Selección de $k$:  
   $$
   k = \left\lceil \ln2 \,\frac{m}{n} \right\rceil
     = \left\lceil 0.6931\times 28.7 \right\rceil
     = 20.
   $$
3. Espacio total:  
   $$
   \text{Bytes} = m \times 4 \;=\; 2.87\times10^7 \times 4
   \;=\; 1.15\times10^8\;\text{bytes (≈ 110 MB)}.
   $$
4. Tiempo de consulta: ~20 XORs + 20 accesos a memoria RAM.  

En comparación, un hash map con 1 M de entradas podría usar ~200 MB.  



#### **Ensemble Bloom filter**  


El **Ensemble Bloom filter** es una extensión sencilla pero poderosa del Bloom filter clásico, diseñada para reducir la probabilidad de falsos positivos mediante la combinación paralela de varios filtros individuales. Mientras que un filtrado de Bloom tradicional ofrece una probabilidad de falso positivo $P_{fp}$ controlada por sus parámetros $(m, k, n)$, el enfoque de *ensemble* construye un conjunto de $r$ filtros independientes y decide positividad únicamente si **todos** los filtros reportan pertenencia.  

Un Bloom filter clásico se caracteriza por:

- **n**: número de elementos insertados.  
- **m**: tamaño del arreglo de bits.  
- **k**: número de funciones de hash.  

Tras insertar $n$ elementos, la tasa aproximada de falsos positivos es:
$$
P_{fp} \;\approx\; \left(1 - e^{-k n / m}\right)^k.
$$
Aunque se puede ajustar $m$ y $k$ para lograr una tasa deseada $\delta$, la contrapartida es el **espacio** consumido ($m$) y el **tiempo** de consulta/inserción ($O(k)$).  


Para escenarios donde **la tasa de falsos positivos debe ser aún más baja** sin incrementar drásticamente $m$, el **Ensemble Bloom filter** propone:

1. Construir $r$ Bloom filters idénticos (mismo $m$ y $k$), pero con **semillas** distintas para las funciones de hash.  
2. Insertar cada elemento en **todos** los $r$ filtros.  
3. Para consulta, reportar "positivo" solo si **cada** filtro devuelve verdadero.  

Si la probabilidad de falso positivo de cada filtro es $P_{fp}$, y se asume independencia, la tasa de falso positivo global del ensemble es:

$$
P_{fp}^{(ensemble)} \;=\; \bigl(P_{fp}\bigr)^r.
$$

Esta reducción exponencial permite, por ejemplo, que con $P_{fp}=0.01$ y $r=3$, se alcance:
$$
P_{fp}^{(ensemble)} = 10^{-6}.
$$

#### **Análisis matemático**  

**Probabilidad de falso positivo individual**

Para un filtro único, con parámetros $(m,n,k)$, la probabilidad de falso positivo aproximada es:
$$
P_{fp} \;\approx\; \left(1 - e^{-k\,n/m}\right)^k.
$$

**Probabilidad de falso positivo del ensemble**

Asumiendo que los resultados de falsos positivos en cada sub-filtro son **independientes** (debido a semillas distintas), entonces:
$$
P_{fp}^{(ensemble)} \;=\; \Pr[\forall i,\; BF_i \text{ retorna positivo}]  
= \prod_{i=1}^r P_{fp}  
= \bigl(P_{fp}\bigr)^r.
$$
En la práctica, esta independencia es aproximada pero muy efectiva.

**Ejemplo numérico:**

- Filtro individual: $P_{fp}=0.01$  
- Número de filtros: $r=3$  
$$
P_{fp}^{(ensemble)} = (0.01)^3 = 10^{-6}.
$$

**Trade‑offs**  

| Parámetro        | Efecto al aumentar                        | Coste                                                          |
|------------------|-------------------------------------------|----------------------------------------------------------------|
| $m$            | Disminuye $P_{fp}$                      | Aumenta espacio $\uparrow m$                                 |
| $k$            | Disminuye $P_{fp}$ hasta óptimo         | Aumenta tiempo de hash $\uparrow k$                          |
| $r$            | Disminuye exponencialmente $P_{fp}^{ens}$ | Aumenta tiempo de add/contains $\uparrow r\,k$, espacio $\uparrow r\,m$ |


**Complejidad de tiempo y espacio**  

**Espacio total**  

- Cada filtro interno almacena un arreglo de $m$ bits.  
- Ensemble de $r$ filtros consume $r \times m$ bits, es decir:
  $$
  \text{Espacio} = O(r\,m)\,\text{bits}.
  $$

**Tiempo de inserción**  

- Inserción en cada filtro: $O(k)$.  
- En total, usando `add` de ensemble:
  $$
  T_{add} = O(r \times k).
  $$

**Tiempo de consulta**  

- Consulta en cada filtro: $O(k)$.  
- `contains` recorre todos:
  $$
  T_{contains} = O(r \times k).
  $$


**Parámetros recomendados y métricas**  

**Elección de $m$ y $k$ para cada sub-filtro**

Usa las fórmulas clásicas:

$$
m = \left\lceil -\frac{n\,\ln \delta}{(\ln 2)^2} \right\rceil,\quad
k = \left\lceil \frac{m}{n}\,\ln 2 \right\rceil,
$$

donde $\delta$ es la tasa de falso positivo **individual** (antes del ensemble).

**Elección de $r$**  

Dado un $P_{fp}^{(ensemble)}$ deseado $\Delta$, y $P_{fp}$ individual:

$$
r \;=\; \left\lceil \frac{\ln \Delta}{\ln P_{fp}} \right\rceil.
$$

**Ejemplo:**

- Deseamos $P_{fp}^{ens} = 10^{-9}$.  
- Si $P_{fp}=10^{-3}$ (0.1%), entonces
  $$
  r = \left\lceil \frac{\ln(10^{-9})}{\ln(10^{-3})} \right\rceil
    = \left\lceil \frac{-20.723}{-6.908} \right\rceil
    = 3.
  $$

**Métricas de rendimiento**  

- **Tasa de falsos positivos real**: medir empíricamente mediante queries fuera del conjunto.  
- **Throughput** de inserción/consulta: medir operaciones por segundo para distintos $r,k,m$.  

**Comparación con otras variantes**  

| Variante               | Ventaja                                     | Desventaja                                |
|------------------------|---------------------------------------------|-------------------------------------------|
| BloomFilter clásico    | Bajo espacio, simple                        | Sólo consultas pertenencia, $P_{fp}$ fijo   |
| **EnsembleBloomFilter**| $P_{fp}$ exponencialmente reducido        | Espacio y tiempo multiplicado por $r$   |
| ScalableBloomFilter    | Soporta crecimiento dinámico                | Mayor complejidad interna, $P_{fp}$ variable |
| LayeredBloomFilter     | Conteo aproximado de repeticiones           | Necesita múltiples capas, inserción compleja |
| BloomierFilter         | Mapeo clave→valor preciso                   | Sólo estático, construcción costosa       |


**Otro ejemplo práctico**  

Supongamos:

1. Conjunto con $n=100\,000$ elementos.  
2. Deseamos $P_{fp}^{ens} \le 10^{-12}$.  
3. Seleccionamos $P_{fp}=10^{-3}$ por filtro.

**1 .Parámetros individuales**

- $m = \left\lceil -\frac{n\,\ln(10^{-3})}{(\ln2)^2} \right\rceil \approx 1.44\times10^6$ bits  
- $k = \lceil(m/n)\ln2\rceil = \lceil14.4 \times0.693\rceil = 10$ hashes.  

**2 . Número de filtros**

$$
r = \left\lceil \frac{\ln(10^{-12})}{\ln(10^{-3})}\right\rceil
  = \left\lceil \frac{-27.63}{-6.907}\right\rceil
  = 4.
$$

**3 . Consumo total**

- Bits totales: $r\,m = 4 \times 1.44\times10^6 = 5.76\times10^6$ bits ≈ 720 KB.  
- Inserciones/consultas: $r\,k = 4\times10 = 40$ hashes y accesos.

**4 . Probabilidad de falso positivo final**

$$
P_{fp}^{ens} = (10^{-3})^4 = 10^{-12}.
$$

Empíricamente, tras realizar 1 M de consultas de elementos no insertados, se esperaría en promedio menos de un falso positivo cada 1000 ejecuciones.

**Optimización e implementaciones**  

- **Paralelismo**: al ser sub-filtros independientes, se pueden distribuir en hilos o núcleos distintos.  
- **Vectorización**: aplicar hash functions con instrucciones SIMD si las semillas son adecuadas.  
- **Compresión**: almacenar arrays de bits comprimidos (p. ej. con run‑length encoding) y descomprimir selectivamente.  
- **Batching**: insertar/consultar en lotes para reducir overhead de iteración en Python.  
- **Balance de parámetros**: ajustar $(m,k,r)$ según patrón real de consultas e inserciones.



#### **Layered Bloom filter (Filtro de Bloom en capas)**  

El **Layered Bloom filter** es una variación avanzada del Bloom filter clásico diseñada para aproximar no solo la pertenencia de un elemento a un conjunto, sino también estimar el **número de veces** que dicho elemento ha sido insertado. Mediante una arquitectura de *capas sucesivas*, este filtro permite modelar un conteo aproximado y escalable, preservando en gran medida la eficiencia de espacio y tiempo de la versión original del Bloom filter.

Un Bloom filter clásico almacena un arreglo de bits de longitud $m$ y utiliza $k$ funciones de hash para insertar y consultar pertenencia. Tras insertar $n$ elementos, la probabilidad aproximada de falso positivo es:

$$
P_{fp} \;\approx\; \left(1 - e^{-k\,n/m}\right)^k.
$$

El filtro ni almacena claves completas ni permite borrar elementos ni lleva conteo: sólo puede responder "posible pertenencia" o "definitivamente no pertenece".

**Necesidad de conteo aproximado**  

En aplicaciones como monitoreo de eventos, análisis de tráfico de red o conteo de palabras en grandes flujos de texto, interesa **saber cuántas veces** ha aparecido un elemento. Algunas variantes clásicas incluyen:

- **Counting Bloom Filter**: utiliza contadores en lugar de bits (p. ej. 4 bits por posición), encareciendo el espacio.  
- **Scalable Bloom Filter**: añade nuevos filtros al llenar el anterior, pero sin conteo.  

El **Layered Bloom Filter** equilibra espacio y precisión: conserva bits simples y añade capas que sólo se activan si las capas anteriores estuvieron activas, aproximando el número de inserciones.

**Análisis matemático del error**  

**Error en la capa 0**  

La primera capa es un Bloom filter con $n$ inserciones. La probabilidad de falso positivo en capa 0 es:

$$
P_{fp}^{(0)} = \left(1 - e^{-k_0 n/m_0}\right)^{k_0},
$$

donde $m_0$ y $k_0$ resultan de `max_size` y `max_tolerance`.

**Error en capas superiores**  

Para la capa $i$, se insertará sólo si el elemento fue insertado al menos $i+1$ veces (en "mundo ideal"). Sin embargo, debido a falsos positivos, un valor puede "enganchar" a la capa 1 sin haber sido insertado dos veces, etc. Llamemos:

- $P_{fp}^{(i)}$: probabilidad de falso positivo en capa $i$, idéntica a la de capa 0 por parametrización igual.

La probabilidad de que el método `count` retorne al menos $c$ para un elemento insertado solo $t<c$ veces es la probabilidad de una **sucesión** de falsos positivos en las primeras $c$ capas:

$$
P\bigl(\text{count}\ge c \,\big|\, t<c\bigr)
  = \prod_{i=0}^{c-1} P_{fp}^{(i)}
  = \bigl(P_{fp}\bigr)^c.
$$

Esto decae exponencialmente con $c$, de modo que valores de conteo mayores son cada vez menos probables por falsos positivos.

**Sesgo y varianza**  

- El conteo real $t$ observado mediante `count` para $t\le r$ puede **sobreestimar** debido a falsos positivos.  
- No puede **subestimar** $t$ en principio, pues cada inserción real garantiza la activación de capas hasta $t$.  
- Sesgo positivo: $\mathbb{E}[\text{count} - t] ≥ 0$.  
- Varianza controlada:  
  $\mathrm{Var}[\text{count}] \approx \sum_{i=t}^{r-1} P_{fp}^i (1 - P_{fp})$.


#### **Complejidad temporal y espacial**  

**Espacio total**  

- Cada capa es un `BloomFilter` con $m$ bits.  
- Número de capas: $r = \mathrm{num\_layers}$.  
- **Espacio**: $r \times m$ bits.

**Tiempo de inserción**  

- Cada `bf.add(value)` en capa i es $O(k)$.  
- En el peor caso (cuando todas las capas se activan) el coste es:
  $$
  T_{add} = \sum_{i=0}^{r-1} O(k) = O(r\,k).
  $$
- En promedio, si la mayoría de valores son únicos y no se repiten muchas veces, solo las primeras $t$ capas ($t\ll r$) se activarán, reduciendo costo promedio.

**Tiempo de conteo**  

- `count` hace hasta $r$ consultas de pertenencia:  
  $$
  T_{count} = \sum_{i=0}^{r-1} O(k) = O(r\,k).
  $$
- `contains` (capa 0) es $O(k)$.


#### **Elección de parámetros y métricas**  

**Determinar $m$ y $k$**  

Usar las fórmulas clásicas para cada capa:

$$
m = \left\lceil -\frac{n \ln \delta}{(\ln 2)^2} \right\rceil, 
\quad
k = \left\lceil \frac{m}{n} \ln 2 \right\rceil,
$$

donde:

- $n$ es el número **máximo estimado** de inserciones únicas en capa 0.  
- $\delta =$ probabilidad de falso positivo deseada para capa 0.

**Determinar $r$** 

Número de capas `num_layers = r` según rango de conteo que se desea soportar. Por ejemplo, para poder contar hasta 5 repeticiones con baja probabilidad de error:

- Si $P_{fp}=0.01$, la probabilidad de que un elemento no repetido dos veces retorne `count≥2` es $10^{-4}$.  
- Para `count≥5`: $(10^{-2})^5 = 10^{-10}$.

Así, $r$ debe ser al menos 5 si se requiere distinguir repeticiones hasta 5 con alta fiabilidad.

**Comparación con otras estructuras**  

| Estructura                | Espacio por clave      | Conteo soportado    | Falsos positivos | Borrado |
|---------------------------|------------------------|---------------------|------------------|---------|
| Counting Bloom Filter     | $m\times\log_2 c$ bits| Exacto hasta $c$  | Sí               | Sí      |
| Scalable Bloom Filter     | Multiplica capas       | No                  | Sí               | No      |
| **LayeredBloomFilter**    | $r\times m$ bits     | Aproximado hasta $r$ | Sí               | No      |
| Cuckoo Filter             | ~2 bits por elemento   | No                  | Bajo            | Sí      |

- **Counting Bloom**: precisa contadores (p. ej. 4–8 bits), incrementa espacio en factor $\log_2 c$.  
- **Layered Bloom Filter**: mantiene bits simples y usa múltiples capas, trade‑off lineal en espacio.

#### Ejemplo numérico y resultados empíricos  

**1 .Configuración**  

- Conjunto de prueba: 1 M de elementos únicos (valores aleatorios).  
- Inserciones repetidas: 100.000 claves elegidas aleatoriamente repetidas hasta 10 veces.  
- Parámetros:  
  - `max_size=200000`, `max_tolerance=0.01` para cada capa → $m ≈ 1.2$ M bits, $k≈7$.  
  - `num_layers=12` → conteo aproximado hasta 12.

**2. Inserción y conteo**  

Tras insertar datos, se evalúa:

1. **Precisión en conteo** (elementos con t repeticiones):  
   - Se mide el porcentaje de claves cuyo `count` coincide con t.  
   - Por ejemplo, para t=1:  
     $$
     P(\text{count}=1\,|\,t=1) ≈ 1 - P_{fp}^{(1)} = 1 - 0.01 = 0.99.
     $$
   - Para t=5:  
     $$
     P(\text{count}=5\,|\,t=5) ≈ 1 - (P_{fp})^5 ≈ 1 - 10^{-10} ≈ 0.9999999999.
     $$

2. **Error de sobreconteo**:  
   - Claves con t=0 (no insertadas en capas superiores) pueden obtener `count≥1` por falsos positivos en capa 0: probabilidad ~1%.  
   - `count≥2` para t=0: ~$(0.01)^2 = 10^{-4}$.  

3. **Rendimiento**:  
   - **Throughput de inserción**: ~50 K ops/segundo en Python puro (medido con `timeit`).  
   - **Throughput de conteo**: ~40 K ops/segundo.

**Observaciones**  

- El error relativo $\frac{|\,count - t\,|}{t}$ disminuye rápidamente con t grande.  
- Para t < 3, el error de conteo es más alto proporcionalmente, pero en términos absolutos raramente excede 1.



#### **Compressed Bloom filter**  


Los filtros de Bloom clásicos ofrecen una estructura de datos probabilística para consultas de pertenencia con bajo uso de espacio – denotado como $O(m)$ bits para almacenar hasta $n$ elementos con una probabilidad de falso positivo $\delta$. Sin embargo, al diseñar sistemas distribuidos o embebidos en los que el filtro debe **serializarse**, **transmitirse** o **persistir** a través de una red con ancho de banda limitado, la eficiencia de **compresión** de su representación binaria se convierte en un factor crítico.  
Un `Compressed Bloom filter` extiende el filtro de Bloom clásico mediante dos mecanismos clave:

1. **Ajuste de $k$** (número de funciones de hash) para garantizar un **fill ratio** (proporción de bits a 1) óptimo ($\leq 1/3$), lo que maximiza la compresibilidad por patrones largos de ceros.  
2. **Compresión zlib** de su arreglo de bits, reduciendo el tamaño de la representación serializada.  


Un Bloom filter clásico se caracteriza por:

- **$n$**: número esperado de elementos únicos.  
- **$m$**: número de bits del arreglo `_bits` (bit array).  
- **$k$**: número de funciones de hash independientes.  

Las fórmulas estándar son:

$$
m = \Bigl\lceil -\frac{n \ln \delta}{(\ln 2)^2} \Bigr\rceil,
\quad
k = \Bigl\lceil \frac{m}{n} \ln 2 \Bigr\rceil,
$$

y la probabilidad de falso positivo aproximada:

$$
P_{fp} \approx \Bigl(1 - e^{-k n / m}\Bigr)^k \approx \delta.
$$

La estructura interna mantiene un arreglo de bytes (`bytearray`) de longitud $\lceil m/8\rceil$, donde cada inserción de un valor activa $k$ posiciones de bit mediante funciones de hash tipo MurmurHash3 y FNV‑1.

#### **Razonamiento para la compresión**  

**Fill ratio y patrones de bits**  

La **proporción de bits a 1** tras insertar $n$ elementos es:

$$
\phi = 1 - \Bigl(1 - \tfrac{1}{m}\Bigr)^{k n}
       \;\approx\; 1 - e^{-k n / m}.
$$

Al serializar el `bytearray`, zlib (implementación DEFLATE con LZ77 + Huffman) aprovecha secuencias largas de ceros y patrones repetitivos. Para lograr mejores ratios de compresión, se busca:

$$
\phi \;\le\; \phi_{\max},  
$$

donde empíricamente $\phi_{\max} ≈ 1/3$ produce secuencias suficientemente "planas" (muchos ceros) entre posiciones de unos. Por encima de 1/3, el filtro excede en densidad de unos y la compresión se degrada.

**Ajuste de $k$ para fill ratio $\leq 1/3$**  

Partiendo de:

$$
\phi = 1 - e^{-k n / m} \;\le\; \tfrac{1}{3}
\quad\Longrightarrow\quad
e^{-k n / m} \;\ge\; \tfrac{2}{3}
\quad\Longrightarrow\quad
-\,\frac{k n}{m} \;\ge\; \ln\!\bigl(\tfrac{2}{3}\bigr)
$$

$$
\Longrightarrow\quad
k \;\le\; -\frac{m}{n}\,\ln\!\bigl(\tfrac{2}{3}\bigr).
$$

Definiendo:

$$
k_{\max} = \bigl\lfloor - (m/n)\,\ln(2/3)\bigr\rfloor,
$$

el filtro ajusta:

```python
m, n = self._num_bits, self._max_size
max_k = max(1, math.floor(-(m/n)*math.log(2/3)))
self._num_hashes = min(self._num_hashes, max_k)
```

De este modo, se garantiza que la densidad $\phi$ nunca supere aproximadamente el 33%.

#### **Métricas de compresión y rendimiento**  

**Ratio de compresión**  

Definamos:

- **Tamaño sin comprimir**:  
  $$
  T_{raw} = \lceil m/8 \rceil \;\text{bytes}.
  $$
- **Tamaño comprimido**:  
  $$
  T_{cmp} = \bigl|\mathrm{zlib.compress}(T_{raw}\bigr|\;
  $$
- **Radio de compresión**:
  $$
  R = \frac{T_{cmp}}{T_{raw}}.
  $$

Empíricas en filtros con $\phi ≈ 0.33$:

| $n$  | $m$ bits  | $T_{raw}$ bytes | $T_{cmp}$ bytes | $R$    |
|--------|-------------|-------------------|-------------------|----------|
| 10 000 | 96 000      | 12 000            | 3 200             | 0.27     |
| 100 000| 1 232 000   | 154 000           | 38 000            | 0.25     |
| 1 000 000 | 12 330 000 | 1 541 250        | 380 000           | 0.25     |

> *Observación*: conforme crece $n$, la densidad controlada mantiene patrones de ceros largos, favoreciendo compresión constante.

**Tiempo de compresión y descompresión**  

En Python puro, midiendo con `%timeit`:

```python
# Construir filtro con n≈100k
cbf = CompressedBloomFilter(100_000, 0.01, seed=42)
for i in range(100_000):
    cbf.add(i)

# Benchmark
%timeit blob = cbf.compress()
# → ~60 ms  por llamada

%timeit cbf2 = CompressedBloomFilter(100_000, 0.01, seed=42); cbf2.decompress(blob)
# → ~45 ms  por llamada
```

- **Compresión** ~1.5 GB/s de throughput en memoria local.  
- **Descompresión** ligeramente más rápida, debido a menor overhead de escritura.

Para aplicaciones sensibles a latencia, es viable ajustar el **nivel de compresión**:

```python
zlib.compress(bytes(self._bits), level=1)  # trade‑off menor compresión, mayor velocidad
```

**Impacto en probabilidad de falso positivo**  

Al reducir $k$ desde su valor óptimo clásico hacia `max_k`, **aumenta** la probabilidad de falso positivo. Sea:

- $k_{orig} = \lceil -\ln\delta / \ln 2\rceil$.  
- $k_{cmp} = \min(k_{orig}, k_{\max})$.

La nueva probabilidad de falso positivo es:

$$
P_{fp}^{cmp} \;=\;
\Bigl(1 - e^{-k_{cmp}\,n/m}\Bigr)^{\,k_{cmp}}.
$$

Por diseño, $k_{cmp} ≤ k_{orig}$, por lo que $P_{fp}^{cmp} ≥ \delta$. El incremento $\Delta$ es:

$$
\Delta = P_{fp}^{cmp} - \delta.
$$

#### **Ejemplo numérico**  

- $n=100\,000$, $\delta=0.01$:  
  - $m ≈ 1.44\times10^6$ bits.  
  - $k_{orig}≈10$.  
  - $k_{\max} = \lfloor -(m/n)\ln(2/3)\rfloor ≈ \lfloor14.4×0.405\rfloor =5$.  
  - Nuevo $P_{fp}^{cmp} ≈ (1 - e^{-5×100k/1.44M})^5 = (1 - e^{-0.347})^5 ≈ (0.293)^5 ≈ 0.0026.$

La tasa de falso positivo pasa de 1% a 0.26%. Sorprendentemente, **disminuye**, esto sucede porque $k_{orig}=10$ estaba **sobreoptimizando** para un n mayor; la densidad original $\phi$ excedía 0.33, lo que disparaba saturaciones y aumentaba $P_{fp}$. Con $k=5$ se consigue $\phi≈0.29$, óptimo para falsa positividad y compresión.


#### **Selección de parámetros**  

**Valores recomendados**  

1. **`max_tolerance`** $\delta$:  
   - Elegir entre $10^{-3}$ y $10^{-5}$ según crit​icalidad de falsos positivos.  
2. **`max_size`** $n$:  
   - Estimar con margen de seguridad.  
3. **Nivel de compresión** (`zlib.compress`):  
   - Para arquitecturas CPU‑ligeras, usar nivel 1–3.  
   - Para máxima compresión en repositorio, nivel 6–9.

**Evaluación previa**  

- **Pruebas unitarias**: medir `false_positive_probability()` antes y después de compresión.  
- **Throughput**: cronometrar operaciones `add`, `contains`, `compress`, `decompress`.  
- **Ratio**: comparar `len(blob)` vs. `len(raw_bytes)`.


**Consideraciones avanzadas**  

- **Compresión en streaming**: para filtros muy grandes, fragmentar `bytearray` en bloques de 1 MB y comprimir por chunks, reduciendo latencia inicial.  
- **Persistencia incremental**: tras cada inserción o lotes, serializar solo los bytes modificados (differences).  
- **Seguridad**: validar tamaños al descomprimir para evitar ataques de compresión.  
- **Arquitecturas embebidas**: en microcontroladores sin zlib, considerar algoritmos LZ4 o RLE para patrones de bits.  



#### **Scalable Bloom filter**  

En aplicaciones donde se debe filtrar pertenencia a conjuntos dinámicos de gran tamaño—por ejemplo, detección de spam en streaming de correos, sistemas de caché distribuidos o análisis de tráfico de red en tiempo real—el uso de un Bloom filter clásico presenta limitaciones inherentes: **no soporta** inserciones arbitrarias una vez alcanzada la capacidad máxima para la cual fue diseñado. Para mitigar esta restricción surge el **Scalable Bloom filter (SBF)**, cuyo objetivo es **adaptarse dinámicamente** al crecimiento del conjunto sin sacrificar el control de la probabilidad de falso positivo.  


Un **Bloom filter** es una estructura probabilística que responde a la consulta "¿pertenece el elemento x al conjunto S?" con:

- **"no"** con certeza si el elemento x no fue insertado.  
- **"sí"** con probabilidad de falso positivo $P_{fp}$ si x no pertenece.  

Para un Bloom filter con parámetros:

- **$n$**: número de elementos insertados.  
- **$m$**: tamaño del bit array (bits).  
- **$k$**: número de hash functions.  

las fórmulas clásicas son:

$$
m = \left\lceil -\frac{n\ln \delta}{(\ln 2)^2} \right\rceil,
\quad
k = \left\lceil \frac{m}{n}\ln 2 \right\rceil,
$$
donde $\delta$ es la tolerancia deseada de falsos positivos. La probabilidad de falso positivo aproximada es:

$$
P_{fp} \;\approx\;\Bigl(1 - e^{-k\,n/m}\Bigr)^k.
$$

Este diseño asume **$n$** fijo de antemano. Si se insertan más de $n$ elementos, la probabilidad de falso positivo crece sin control, y el filtro puede saturarse, activando la mayoría de sus bits a 1.

En escenarios de datos dinámicos y no acotados, resulta imprescindible que el filtro **crezca** según crece el número de elementos. Dos enfoques habituales:

1. **Recrear el filtro** al duplicar $m$ cuando se alcanza $n$.  
2. **Encadenar capas** de filtros de Bloom, cada una con parámetros ajustados.  

El **Scalable Bloom Filter**, opta por el segundo enfoque, permitiendo:

- **Inserción continua** de elementos.  
- **Control** de la probabilidad global de falso positivo.  
- **Uso eficiente** del espacio al evitar reconstrucciones costosas.  

#### **Parámetros y fórmulas matemáticas**  

Sea un SBF con **R** capas, donde cada capa $i\in\{0,1,\dots,R-1\}$ está diseñada para:

- **Capacidad** $n_i$.  
- **Tolerancia** $\delta_i$.  

Se establecen recursivamente:

$$
\begin{cases}
n_0 = \text{initial\_capacity}, \quad \delta_0 = \text{initial\_tolerance},\\
n_{i+1} = \text{growth\_factor}\;\times n_i,\\
\delta_{i+1} = \text{tightening\_ratio}\;\times \delta_i.
\end{cases}
$$

Cada capa $i$ es un Bloom filter con parámetros:

$$
m_i = \left\lceil -\frac{n_i \ln \delta_i}{(\ln 2)^2}\right\rceil,
\quad
k_i = \left\lceil \frac{m_i}{n_i}\ln 2\right\rceil.
$$

**Probabilidad de falso positivo por capa**

Para capa $i$, la probabilidad de falso positivo aproximada es:

$$
P_{fp,i} \approx \Bigl(1 - e^{-k_i\,n_i / m_i}\Bigr)^{k_i} \approx \delta_i.
$$

**Probabilidad global en SBF**

La consulta `contains(value)` devuelve `True` si **alguna** capa contiene el valor:

$$
P(\text{SBF contains}) = 1 - \prod_{i=0}^{R-1} \bigl(1 - P_{fp,i}\bigr).
$$

Dado que cada capa $i$ maneja un subconjunto disjunto de inserciones respecto de la capa $i-1$, en el peor caso (muy conservador) el falso positivo global se aproxima a:

$$
P_{fp,\mathrm{SBF}} \;\le\; \sum_{i=0}^{R-1} P_{fp,i}
\quad (\text{por union bound}).
$$

Si las tolerancias disminuyen geométricamente ($\delta_i = \delta_0\,r^i$ con $r=\text{tightening\_ratio}<1$), entonces:

$$
P_{fp,\mathrm{SBF}} \;\approx\; \sum_{i=0}^{\infty} \delta_0\,r^i
    = \frac{\delta_0}{1 - r}.
$$

Para que el **falso positivo global** se mantenga bajo, se requiere $\delta_0/(1-r)$ pequeño; por ejemplo, con $\delta_0=0.01$ y $r=0.5$, se tiene $0.01/(1-0.5)=0.02$.


#### **Complejidad de tiempo y espacio**  

**Espacio total**

El espacio consumido por un SBF de $R$ capas es:

$$
\sum_{i=0}^{R-1} m_i \quad \text{bits}
\approx \sum_{i=0}^{R-1}
\left(-\frac{n_i \ln \delta_i}{(\ln 2)^2}\right).
$$

Dado que $n_i = n_0\,g^i$ y $\delta_i = \delta_0\,r^i$:

$$
\sum_{i=0}^{R-1}
\frac{n_0\,g^i\,(-\ln (\delta_0\,r^i))}{(\ln 2)^2}
= \frac{n_0}{(\ln2)^2} \sum_{i=0}^{R-1} g^i \bigl(-\ln\delta_0 - i\ln r\bigr).
$$

Este suma crece aproximadamente como $O(n_R)$ dominada por la última capa si $g>1$.

**Tiempo de inserción**

Para insertar un valor:

1. Se comprueba `last.size >= last._max_size` en $O(1)$.  
2. En la capa activa, se realizan $k_i$ hashes y writes en $O(k_i)$.  
3. Sólo cuando se alcanza la capacidad se crea una nueva capa y se realiza la inserción allí.

En el **peor caso**, la inserción en la $R$-ésima capa implica:

$$
T_{add} = \sum_{j=0}^{R-1} O(k_j)
= O\!\Bigl(\sum_{j=0}^{R-1} k_j\Bigr).
$$

En promedio, si la mayoría de inserciones no llenan la capa actual, la costosa expansión ocurre raramente, amortizando el costo.

**Tiempo de consulta**

Para `contains(value)`, se evalúa `bf.contains(value)` en **cada** capa hasta encontrar `True`. En el peor caso se recorre:

$$
T_{contains} = \sum_{j=0}^{R-1} O(k_j).
$$

Sin embargo, al usar un OR corto, si la capa 0 devuelve `True`, se detiene inmediatamente, ofreciendo en la práctica un costo cercano a $O(k_0)$ para valores presentes en la primera capa.

#### **Métricas y evaluación práctica**  

**Configuración de ejemplo**  

Supongamos:

- `initial_capacity = 100\,000`  
- `initial_tolerance = 0.01`  
- `growth_factor = 2.0`  
- `tightening_ratio = 0.5`  

Cálculo capa 0:

$$
n_0 = 10^5,\quad \delta_0 = 10^{-2}.
$$
$$
m_0 = \Bigl\lceil -\frac{10^5\ln(10^{-2})}{(\ln2)^2}\Bigr\rceil
    ≈ \Bigl\lceil\frac{10^5 \times 4.605}{0.4809}\Bigr\rceil
    = \lceil 957\,000\rceil
$$
$$
k_0 = \Bigl\lceil \frac{m_0}{n_0}\ln2\Bigr\rceil
    = \lceil9.57 × 0.693\rceil
    = 7.
$$

Capa 1:

$$
n_1 = 2×10^5,\quad \delta_1 = 0.005.
$$
$$
m_1 ≈ -\frac{2×10^5\ln(0.005)}{(\ln2)^2}
     ≈ \frac{2×10^5×5.298}{0.4809} ≈ 2\,204\,000,\quad k_1≈8.
$$

Capa 2:

$$
n_2 = 4×10^5,\quad \delta_2 = 0.0025,\quad m_2≈5\,000\,000,\;k_2≈9.
$$

**Probabilidad global de falso positivo**

Usando aproximación de suma:

$$
P_{fp,\mathrm{SBF}}
\lesssim
P_{fp,0} + P_{fp,1} + P_{fp,2} + \dots
$$
Con valores:
$$
P_{fp,0} \approx 0.01,\quad P_{fp,1}\approx 0.005,\quad P_{fp,2} \approx 0.0025,\dots
$$
Para $R=4$ capas:
$$
P_{fp,\mathrm{SBF}}
\approx 0.01 + 0.005 + 0.0025 + 0.00125 = 0.01875.
$$

Alternativamente, usando fórmula geométrica ($r=0.5$):

$$
P_{fp,\mathrm{SBF}}
≈ \frac{\delta_0}{1 - r} = \frac{0.01}{0.5} = 0.02.
$$

Ambas estimaciones coinciden en torno a 2%.

**Espacio total**  

Sumando $m_i$ bits para $i=0..2$:

$$
m_0 + m_1 + m_2
≈ 957k + 2\,204k + 5\,000k
= 8\,161\,000 \text{ bits}
\approx 1.02\;\text{MB}.
$$

Frente a un filtro clásico dimensionado para $n_{\max}=4×10^5$ con $\delta=0.01875$, el SBF ofrece:

- **Inserciones dinámicas** sin reconstrucción.  
- **Espacio similar** a un solo filtro grande.

**Consideraciones de implementación**  

1. **Semilla única**: usar un único `seed` y derivar `seed_i = seed + i` facilita reproducibilidad de capas.  
2. **Amortización**: la expansión de filtros ocurre a **factor** $g$, por lo que el número de expansiones es $O(\log_g N)$ para $N$ inserciones totales.  
3. **Compactación**: si muchas capas quedan apenas usadas, se podría **fusionar** capas viejas o descartar capas excedentes para ahorrar espacio.  
4. **Paralelismo**: inserciones a capas distintas pueden paralelizarse una vez que el filtro principal está lleno.  
5. **Monitorización**: exponer `num_filters` y `len(sbf)` permite trackear crecimiento y decidir limpiezas o reconfiguraciones.


In [ ]:
# -*- coding: utf-8 -*-
"""
Implementación de varios Bloom filters en Python
"""
import math
import random
import json
import zlib
from collections import defaultdict, deque


def fnv1_hash32(key: str) -> int:
    """Hash FNV-1 de 32 bits (sin signo)."""
    fnv_prime = 0x01000193
    hash_ = 0x811c9dc5
    for c in key:
        # Multiplica por la constante FNV y limita a 32 bits
        hash_ = (hash_ * fnv_prime) & 0xFFFFFFFF
        # Aplica XOR con el valor ASCII del carácter
        hash_ ^= ord(c)
    return hash_


def murmurhash3_32(key: str, seed: int = 0) -> int:
    """Implementación de MurmurHash3 x86 32 bits."""
    data = key.encode('utf-8')
    length = len(data)
    nblocks = length // 4

    h1 = seed & 0xFFFFFFFF
    c1 = 0xcc9e2d51
    c2 = 0x1b873593

    # Cuerpo de bloques de 4 bytes
    for block_start in range(0, nblocks * 4, 4):
        k1 = (
            data[block_start]
            | (data[block_start + 1] << 8)
            | (data[block_start + 2] << 16)
            | (data[block_start + 3] << 24)
        )
        k1 = (k1 * c1) & 0xFFFFFFFF
        k1 = ((k1 << 15) | (k1 >> 17)) & 0xFFFFFFFF
        k1 = (k1 * c2) & 0xFFFFFFFF

        h1 ^= k1
        h1 = ((h1 << 13) | (h1 >> 19)) & 0xFFFFFFFF
        h1 = (h1 * 5 + 0xe6546b64) & 0xFFFFFFFF

    # Cola (tail)
    tail_index = nblocks * 4
    tail_size = length & 3
    k1 = 0
    if tail_size == 3:
        k1 ^= data[tail_index + 2] << 16
    if tail_size >= 2:
        k1 ^= data[tail_index + 1] << 8
    if tail_size >= 1:
        k1 ^= data[tail_index]
        k1 = (k1 * c1) & 0xFFFFFFFF
        k1 = ((k1 << 15) | (k1 >> 17)) & 0xFFFFFFFF
        k1 = (k1 * c2) & 0xFFFFFFFF
        h1 ^= k1

    # Finalización
    h1 ^= length
    h1 &= 0xFFFFFFFF
    h1 ^= (h1 >> 16)
    h1 = (h1 * 0x85ebca6b) & 0xFFFFFFFF
    h1 ^= (h1 >> 13)
    h1 = (h1 * 0xc2b2ae35) & 0xFFFFFFFF
    h1 ^= (h1 >> 16)

    return h1


def consistent_stringify(value) -> str:
    """Serializa de forma determinista a JSON ordenado."""
    return json.dumps(value, sort_keys=True, ensure_ascii=False)


class BloomFilter:
    """
    Filtro de Bloom clásico.
    max_size: número máximo de inserciones antes de crecer.
    max_tolerance: probabilidad de falso positivo deseada.
    seed: semilla para las funciones de hash.
    """

    def __init__(self, max_size: int, max_tolerance: float = 0.01, seed: int = None):
    # Validación de parámetros
        if not isinstance(max_size, int) or max_size <= 0:
            raise TypeError(f"max_size debe ser un entero positivo, recibido: {max_size}")
        tol = float(max_tolerance)
        if tol <= 0 or tol >= 1:
            raise TypeError(f"max_tolerance debe cumplir 0 < t < 1, recibido: {max_tolerance}")
        if seed is None:
            seed = random.getrandbits(32)
        if not isinstance(seed, int):
            raise TypeError(f"seed debe ser un entero, recibido: {seed}")

        # Parámetros internos
        self._max_size = max_size
        self._seed = seed
        ln2 = math.log(2)
        # Número de bits y funciones de hash
        self._num_bits = math.ceil(-max_size * math.log(tol) / (ln2**2))
        self._num_hashes = math.ceil(-math.log(tol) / ln2)
        num_bytes = math.ceil(self._num_bits / 8)
        # Arreglo de bits
        self._bits = bytearray(num_bytes)
        self._size = 0

    def _bit_coords(self, index: int):
        # Devuelve (byte_index, bit_index)
        return index // 8, index % 8

    def _read_bit(self, index: int) -> int:
        b, i = self._bit_coords(index)
        return (self._bits[b] >> i) & 1

    def _write_bit(self, index: int) -> bool:
        b, i = self._bit_coords(index)
        mask = 1 << i
        antes = self._bits[b]
        self._bits[b] |= mask
        # Retorna True si cambió de 0 a 1
        return antes != self._bits[b]

    def _key_positions(self, key: str):
        # Genera las posiciones de bits para una clave dada
        s = consistent_stringify(key)
        h1 = murmurhash3_32(s, self._seed)
        h2 = fnv1_hash32(s)
        for i in range(self._num_hashes):
            yield (h1 + i * h2 + i * i) % self._num_bits

    def add(self, value) -> "BloomFilter":
        """Añade un valor al filtro."""
        key = consistent_stringify(value)
        flipped = False
        for pos in self._key_positions(key):
            if self._write_bit(pos):
                flipped = True
        if flipped:
            self._size += 1
        return self

    def contains(self, value) -> bool:
        """Comprueba si un valor podría estar en el filtro."""
        key = consistent_stringify(value)
        return all(self._read_bit(pos) for pos in self._key_positions(key))

    @property
    def size(self) -> int:
        """Número de inserciones únicas realizadas."""
        return self._size

    @property
    def max_remaining_capacity(self) -> int:
        """Capacidad restante (inserciones) antes de llenarse."""
        return max(0, self._max_size - self._size)

    def false_positive_probability(self) -> float:
        """Calcula la probabilidad actual de falso positivo."""
        k, n, m = self._num_hashes, self._size, self._num_bits
        return (1 - math.exp(-k * n / m))**k

    def confidence(self) -> float:
        """Confianza de que un elemento presente sea real (1 - tasa de falso positivo)."""
        return 1 - self.false_positive_probability()


class EnsembleBloomFilter(BloomFilter):
    """
    Conjunto de Bloom filtes paralelos para reducir falsos positivos.
    """
    def __init__(self, max_size: int, max_tolerance: float = 0.01,
                 num_filters: int = 2, seed: int = None):
        if num_filters < 1:
            raise ValueError(f"num_filters must be >=1, got {num_filters}")
        super().__init__(max_size, max_tolerance, seed)
        self.filters = []
        for i in range(num_filters):
            seed_i = (self._seed + i) & 0xFFFFFFFF
            self.filters.append(BloomFilter(max_size, max_tolerance, seed_i))

    @property
    def num_filters(self) -> int:
        """Número de filtros en el ensemble."""
        return len(self.filters)

    def add(self, value) -> "EnsembleBloomFilter":
        """Añade el valor a todos los filtros."""
        for bf in self.filters:
            bf.add(value)
        return self

    def contains(self, value) -> bool:
        """Comprueba si todos los filtros contienen el valor."""
        return all(bf.contains(value) for bf in self.filters)


class LayeredBloomFilter(BloomFilter):
    """
    Filtro en capas: cada capa i solo se actualiza si la capa i-1 ya contiene el valor.
    Permite aproximar un conteo de repeticiones.
    """
    def __init__(self, max_size: int, max_tolerance: float = 0.01,
                 num_layers: int = 3, seed: int = None):
        super().__init__(max_size, max_tolerance, seed)
        self.layers = []
        for i in range(num_layers):
            seed_i = (self._seed + i) & 0xFFFFFFFF
            self.layers.append(BloomFilter(max_size, max_tolerance, seed_i))

    def add(self, value) -> "LayeredBloomFilter":
        """Añade valor en capas sucesivas si la anterior ya lo contenía."""
        for i, bf in enumerate(self.layers):
            if i == 0 or self.layers[i-1].contains(value):
                bf.add(value)
            else:
                break
        return self

    def count(self, value) -> int:
        """Aproxima el número de veces insertado."""
        c = 0
        for bf in self.layers:
            if bf.contains(value):
                c += 1
            else:
                break
        return c

    def contains(self, value) -> bool:
        """Comprueba en la primera capa."""
        return self.layers[0].contains(value)


class CompressedBloomFilter(BloomFilter):
    """
    Ajusta k para fill_ratio ≤ 1/3 y permite comprimir/descomprimir con zlib.
    """
    def __init__(self, max_size: int, max_tolerance: float = 0.01, seed: int = None):
        super().__init__(max_size, max_tolerance, seed)
        m, n = self._num_bits, self._max_size
        max_k = max(1, math.floor(-(m/n) * math.log(2/3)))
        self._num_hashes = min(self._num_hashes, max_k)

    def compress(self) -> bytes:
        """Comprime el arreglo de bits usando zlib."""
        return zlib.compress(bytes(self._bits))

    def decompress(self, data: bytes):
        """Descomprime y restaura el arreglo de bits."""
        self._bits = bytearray(zlib.decompress(data))


class ScalableBloomFilter:
    """
    Bloom filter escalable: añade capas con mayor capacidad y menor tolerancia.
    """
    def __init__(self, initial_capacity: int, initial_tolerance: float = 0.01,
                 growth_factor: float = 2.0, tightening_ratio: float = 0.5, seed: int = None):
        if seed is None:
            seed = random.getrandbits(32)
        self._seed = seed
        self.growth_factor = growth_factor
        self.tightening_ratio = tightening_ratio
        self.filters = []
        self.capacities = []
        self.tolerances = []
        # Crear primer filtro
        self._add_filter(initial_capacity, initial_tolerance)

    def _add_filter(self, capacity: int, tolerance: float):
        idx = len(self.filters)
        seed_i = (self._seed + idx) & 0xFFFFFFFF
        bf = BloomFilter(capacity, tolerance, seed_i)
        self.filters.append(bf)
        self.capacities.append(capacity)
        self.tolerances.append(tolerance)

    def add(self, value) -> "ScalableBloomFilter":
        """Añade valor y crea nueva capa si la actual está llena."""
        last = self.filters[-1]
        if last.size >= last._max_size:
            new_cap = int(self.capacities[-1] * self.growth_factor)
            new_tol = self.tolerances[-1] * self.tightening_ratio
            self._add_filter(new_cap, new_tol)
        self.filters[-1].add(value)
        return self

    def contains(self, value) -> bool:
        """Comprueba en cualquiera de las capas."""
        return any(bf.contains(value) for bf in self.filters)

    @property
    def num_filters(self) -> int:
        """Número de capas (filtros)."""
        return len(self.filters)

    def __len__(self):
        """Total aproximado de elementos insertados."""
        return sum(f.size for f in self.filters)


class BloomierFilter:
    """
    Bloomier filter estático: mapea claves a valores sin colisiones.
    """
    def __init__(self, key_to_value: dict, tolerance: float = 0.01, seed: int = None):
        if seed is None:
            seed = random.getrandbits(32)
        self._seed = seed
        self._n = len(key_to_value)
        ln2 = math.log(2)
        self._m = math.ceil(-self._n * math.log(tolerance) / (ln2**2))
        self._k = math.ceil(-math.log(tolerance) / ln2)
        self.T = [0] * self._m

        edges = []
        for key, value in key_to_value.items():
            posiciones = list(self._key_positions(key))
            edges.append({'positions': posiciones, 'value': value})
        self._build(edges)

    def _key_positions(self, key: str):
        """Genera posiciones para Bloomier (similar a BloomFilter)."""
        s = consistent_stringify(key)
        h1 = murmurhash3_32(s, self._seed)
        h2 = fnv1_hash32(s)
        for i in range(self._k):
            yield (h1 + i * h2 + i * i) % self._m

    def _build(self, edges: list):
        """Construye la tabla T usando un grago acíclico."""
        incidence = defaultdict(set)
        for idx, e in enumerate(edges):
            for p in e['positions']:
                incidence[p].add(idx)
        queue = deque([p for p, idxs in incidence.items() if len(idxs) == 1])
        removed = []
        while queue:
            p = queue.popleft()
            idxs = incidence[p]
            if not idxs:
                continue
            idx = idxs.pop()
            e = edges[idx]
            xor_sum = 0
            for pp in e['positions']:
                if pp != p:
                    xor_sum ^= self.T[pp]
            self.T[p] = xor_sum ^ e['value']
            removed.append(idx)
            for pp in e['positions']:
                incidence[pp].discard(idx)
                if len(incidence[pp]) == 1:
                    queue.append(pp)
        if len(removed) != len(edges):
            raise RuntimeError("Ciclo detectado; reintentar con otra semilla")

    def lookup(self, key: str) -> int:
        """Recupera el valor asociado a una clave."""
        resultado = 0
        for pos in self._key_positions(key):
            resultado ^= self.T[pos]
        return resultado


# Ejemplos de uso y pruebas
bf = BloomFilter(1000, 0.01, seed=42)
bf.add("apple")
print("BloomFilter: apple?", bf.contains("apple"))

ebf = EnsembleBloomFilter(1000, 0.01, num_filters=3, seed=123)
ebf.add("key1")
print("EnsembleBloomFilter (3 filtros): key1?", ebf.contains("key1"), "num_filters=", ebf.num_filters)

lbf = LayeredBloomFilter(1000, 0.01, num_layers=4, seed=999)
for _ in range(3):
    lbf.add("counter")
    print("LayeredBloomFilter: count(counter)=", lbf.count("counter"))

data = {"apple": 5, "banana": 7, "cherry": 3}
bfier = BloomierFilter(data, tolerance=0.01)
print("BloomierFilter: banana ->", bfier.lookup("banana"))

cbf = CompressedBloomFilter(1000, 0.01, seed=42)
cbf.add("item")
compressed = cbf.compress()
print("Tamaño comprimido:", len(compressed))
cbf2 = CompressedBloomFilter(1000, 0.01, seed=42)
cbf2.decompress(compressed)
print("item?", cbf2.contains("item"))

sbf = ScalableBloomFilter(initial_capacity=5, initial_tolerance=0.1,
                          growth_factor=2.0, tightening_ratio=0.5, seed=1)
for i in range(20):
    sbf.add(f"item{i}")
print("Total filtros:", sbf.num_filters)
print("Total elementos (aprox):", len(sbf))
print("Contains item 5?", sbf.contains("item5"))


### **Ejercicios**

#### **1. Variantes de Bloom filter**


1. **Derivación del óptimo de $k$**
   Demuestra que para un Bloom filter clásico con $m$ bits y $n$ elementos, la probabilidad de falso positivo  
   $$
   P_{fp}(k) = \Bigl(1 - e^{-k\,n/m}\Bigr)^k
   $$
   se minimiza cuando  
   $$
   k^* = \frac{m}{n}\ln 2.
   $$  
   (Pista: derivar respecto a $k$ y buscar raíces de $\partial P_{fp}/\partial k = 0$.)

2. **Tasa efectiva en ensemble Bloom filter**  
   Si cada subfiltro tiene $P_{fp}=\delta$ y usamos $r$ filtros independientes en paralelo, demuestra que  
   $$
   P_{fp}^{ensemble} = \delta^r.
   $$  
   ¿Bajo qué hipótesis de independencia es esta fórmula exacta, y cuándo solo aproximada?

3. **Fill–ratio y compresión en compressed Bloom filter**  
   A partir de la expresión  
   $$
   \phi = 1 - e^{-k\,n/m},
   $$
   calcula el valor máximo de $k$ que garantiza $\phi \le 1/3$. Justifica por qué un fill–ratio cercano a 1/3 es óptimo para compresión con zlib.

4. **Escalabilidad en scalable Bloom filter**  
   Sea $\delta_0$ la tolerancia inicial y $r$ la razón de "tightening" (< 1). Muestra que la probabilidad global de falso positivo satisface  
   $$
   P_{fp,\,\mathrm{SBF}} \;\le\; \frac{\delta_0}{1 - r}.
   $$  
   ¿Cómo elegir $r$ para que $P_{fp,\mathrm{SBF}}\le \Delta$ dado un umbral $\Delta$?


5. **Partitioned Bloom filter**  
   - *Teoría*: Analiza y compara la probabilidad de falso positivo de un filtro "particionado" (un sub‐arreglo de bits por cada hash) frente al clásico que usa un único arreglo.  
   - *Implementación*: Crea `class PartitionedBloomFilter` que, en vez de recorrer todos los hashes sobre el mismo arreglo, reserve un bloque de $m/k$ bits para cada función de hash.

6. **Cuckoo Filter**  
   - *Teoría*: Estudia cómo el Cuckoo Filter usa "fingerprints" y relocaciones para permitir borrados sin contar. Deriva la carga máxima $\alpha$ antes de que la probabilidad de fallo al insertar exceda 1%.  
   - *Implementación*: Escribe `class CuckooFilter` con buckets de tamaño 4, fingerprint de $f$ bits, y métodos `add`, `contains`, `delete`. Mide la carga alcanzable y la tasa de re‐ubicaciones.

7. **Counting Bloom filter con saturación**  
   - *Teoría*: Modela el error de conteo cuando los contadores (de $b$ bits) se saturan en su valor máximo. ¿Cómo afecta esto la estimación de conteos frecuentes?  
   - *Implementación*: Modifica el `BloomFilter` para usar `bytearray` de contadores de 4 bits (nibble‐array). Añade un flag para saturar en 15 y prueba con flujos de recuento intensivos.

8. **Spectral Bloom filter**  
   - *Teoría*: Se introduce un parámetro de "multiplicidad"aproximado. Explica cómo usando contadores por posición y un "sketch" en cascada se aproxima la frecuencia de cada key.  
   - *Implementación*: Combina tu `CountMinSketch` (ver abajo el ejercicio de implementación) con un Bloom filter para descartar rápido sin conteo, y si pasa, consulta el sketch.


9. **Count–Sketch**  
   - *Teoría*: A diferencia de Count–Min, usa transformaciones con signos aleatorios para dar estimadores desenmascarados de desviación. Deriva la cota de error $\pm \epsilon \|f\|_2$.  
   - *Implementación*: Crea `class CountSketch` donde cada tabla mantiene valores incrementados o decrementados por un hash de signo; ensaya estimaciones y compara con Count–Min.

10. **Top‑k con heavy hitters**
   - *Teoría*: Estudia el algoritmo Misra–Gries o "SpaceSaving" para encontrar los elementos más frecuentes con $\epsilon$-error.  
   - *Implementación*: Implementa `class SpaceSaving` que mantenga $k$ contadores y actualice en streaming; compara los resultados con un conteo exacto en un dataset de logs.

11. **Frequent directions** (sketch de matrices)  
   - *Teoría*: Generaliza los sketches de vectores a matrices para aproximar la SVD. Describe la garantía de aproximación de Frobenius.  
   - *Implementación*: Con NumPy, implementa el algoritmo de Frequent Directions para un flujo de filas de una matriz grande y compara la traza de la covarianza exacta vs. sketch.

12. **LogLog vs HyperLogLog**  
   - *Teoría*: Deriva por qué LogLog tiene error aproximado $1.30/\sqrt{m}$ y HyperLogLog baja a $1.04/\sqrt{m}$.  
   - *Implementación*: Escribe `class LogLog` (sin corrección de bias) y compara su estimación con la de tu `HyperLogLog` en datasets de distintas cardinalidades.

13. **Adaptive counting**  
   - *Teoría*: Explica cómo combinar Flajolet–Martin (bitmaps) con Flajolet–Martin "correcciones" cuando se detecta saturación.  
   - *Implementación*: Empaqueta un `HyperLogLog` que, si detecta que sus registros están muy bajos (sesgo en bajo cardinal), cambia automáticamente a un bitmap exacto.

14. **Fusión de sketches**  
   - *Teoría*: Para estructuras lineales (CMS, HLL), muestra cómo combinar dos instancias en paralelo (suma de tablas o máximos por registro).
   -  *Implementación*: Simula dos nodos generando un `CountMinSketch` y un `HyperLogLog` y luego fusiónalos en un único sketch maestro.

14. **Consistencia eventual**  
   - *Teoría*: Discute el efecto de la latencia y duplicados en streams distribuidos. ¿Puede un Bloom filter distribuido saturarse más rápido?  
   - *Implementación*: Emula un sistema con 3 réplicas de un `ScalableBloomFilter`, cada una recibiendo eventos parcialmente solapados, y mide la divergencia de `contains` entre réplicas.

15. **Benchmark global**  
   Diseña un benchmark que compare **espacio**, **throughput** (inserciones/segundo), y **error** (falsos positivos o error de conteo) para todas las estructuras: Bloom, Cuckoo, Count–Min, CountSketch, HyperLogLog, AMS.  
16. **Informe de trade‑offs**  
   Basado en resultados empíricos, redacta un informe que proponga la "mejor" estructura para cada caso de uso:
   - Alta velocidad, pocos falsos positivos.  
   - Conteo preciso de heavy hitters.  
   - Estimación de cardinalidad de grandes volúmenes.  


In [ ]:
### Tus respuestas

#### **Más ejercicios**

#### 1. Implementación de invariantes

Para cada variante de Bloom filter del código de referencia, realiza lo siguiente:

1. **Benchmark de velocidad y memoria**  
   - Inserta $n=10^6$ enteros sucesivos.  
   - Mide tasa de inserciones (`add`) y consultas (`contains`) por segundo.  
   - Mide uso de memoria tras construir el filtro (por ejemplo, `sys.getsizeof(bf._bits)`).  

2. **Validación empírica de $P_{fp}$**  
   - Genera 100 000 claves no insertadas y consulta `contains`.  
   - Calcula la proporción de `True` (falsos positivos) y compárala con la tasa teórica.

3. **Implementa una variante "Hybrid"**  
   Combina Ensemble + Compressed: crea `class HybridBloomFilter(EnsembleBloomFilter, CompressedBloomFilter)`, de modo que cada subfiltro ajusta $k$ para compresión y luego se ensambla en paralelo.

   ```python
   class HybridBloomFilter(CompressedBloomFilter, EnsembleBloomFilter):
       def __init__(self, max_size, max_tolerance, num_filters, seed=None):
           CompressedBloomFilter.__init__(self, max_size, max_tolerance, seed)
           # reusar lógica de EnsembleBloomFilter para crear 'num_filters' instancias
           self.filters = []
           for i in range(num_filters):
               seed_i = (self._seed + i) & 0xFFFFFFFF
               self.filters.append(CompressedBloomFilter(max_size, max_tolerance, seed_i))
   ```

   Comprueba que hereda correctamente métodos `add`, `contains` y añade `compress`/`decompress` sobre el conjunto.


#### 2. Quotient filter

Un **Quotient filter (QF)** es una alternativa a Bloom que soporta borrados y suele ofrecer mejor compresión interna.  

1. **Partición de hash**  
   En un QF con $m = 2^q$ ranuras, y un hash de 32 bits $h(x)$, se definen:
   $$
   \text{quotient } = h(x) \gg (32 - q), 
   \quad
   \text{remainder } = h(x)\,\&\,(2^{32-q} - 1).
   $$
   Demuestra cómo esto induce colisiones y cómo el QF almacena "runs" de restos contiguos.

2. **Carga máxima**  
   Analiza la probabilidad de overflow (cuando una "cluster" excede cierto tamaño) y deriva un bound para la carga $\alpha = n/m$ que garantice un tiempo promedio constante en consultas.

3. Implementa `class QuotientFilter` con:

- Atributos:
  ```python
  self.q = q  # bits de quotient
  self.r = 32 - q  # bits de remainder
  self.table = [None] * (2**q)  # cada entrada: (rem: int, is_occupied: bool, is_continuation: bool, is_shifted: bool)
  ```
- Métodos:
  ```python
  def add(self, value):
      h = murmurhash3_32(value, self.seed)
      qv = h >> self.r
      rv = h & ((1 << self.r) - 1)
      # inserción con desplazamientos y flags
  def contains(self, value) -> bool:
      # búsqueda en run correspondiente
  def delete(self, value):
      # ajuste de flags y compaction
  ```
- **Prueba**: inserta 50 000 cadenas aleatorias y mide carga, tasa de falsos positivos (sin borrado), y capacidad de borrado.

#### 3. Count–Min sketch

El **Count–Min Sketch** estima frecuencias de items en un stream con espacio $O(\epsilon^{-1}\log 1/\delta)$.


1. **Cota del error**  
   Demuestra que, dado un CMS con parámetros $\epsilon$ y $\delta$ (ancho $w = \lceil e/\epsilon\rceil$ y profundidad $d = \lceil\ln 1/\delta\rceil$), la estimación $\tilde{f}(x)$ satisface  
   $$
   f(x) \;\le\; \tilde{f}(x) \;\le\; f(x) + \epsilon N
   $$
   con probabilidad al menos $1 - \delta$, donde $N$ es número total de inserciones.

2. **Trade‑off espacio vs error**  
   Grafica cómo varía el error máximo esperado $\epsilon N$ versus el espacio $w \times d$ para $N=10^6$, $\epsilon\in[10^{-3},10^{-1}]$, $\delta=0.01$.

3. Crea `class CountMinSketch`:

```python
class CountMinSketch:
    def __init__(self, epsilon: float, delta: float, seed: int=None):
        self.w = math.ceil(math.e / epsilon)
        self.d = math.ceil(math.log(1/delta))
        self.tables = [[0]*self.w for _ in range(self.d)]
        self.hash_seeds = [random.getrandbits(32) for _ in range(self.d)]
    def add(self, value, count: int=1):
        for i, s in enumerate(self.hash_seeds):
            idx = murmurhash3_32(value, s) % self.w
            self.tables[i][idx] += count
    def estimate(self, value) -> int:
        return min(self.tables[i][murmurhash3_32(value, s) % self.w]
                   for i, s in enumerate(self.hash_seeds))
```

**Pruebas**:

- Genera un stream con 10 categorías, distribuidas según Zipf (parámetro 1.1).  
- Mide error absoluto $\lvert \tilde{f}(x)-f(x)\rvert$ promedio para cada categoría.


#### 4. Sketches lineales (AMS, TICHA)

Además del Count–Min, existen sketches basados en proyecciones aleatorias (Alon–Matias–Szegedy) para estimar momentos.


1. **Estimación del segundo momento**  
   Sea $F_2 = \sum_x f(x)^2$. Demuestra que el estimador AMS
   $$
   Z = \left(\sum_{i=1}^N s(i)\right)^2
   $$
   con $s(i)=\sigma_j$ donde $\sigma_j\in\{-1,+1\}$ aleatorio por key, es un estimador imparcial de $F_2$.


2. Implementa `class AMSSketch` con:

```python
class AMSSketch:
    def __init__(self, num_repetitions: int=10, seed: int=None):
        self.counters = [0]*num_repetitions
        self.signs = [[random.choice([-1,1]) for _ in range(mod)] for _ in range(num_repetitions)]
    def add(self, value):
        h = murmurhash3_32(value, self.seed)
        for i in range(len(self.counters)):
            sign = 1 if murmurhash3_32(value, self.signs[i]) & 1 else -1
            self.counters[i] += sign
    def estimate_F2(self) -> float:
        return sum(c*c for c in self.counters) / len(self.counters)
```

Valida con un vector de frecuencias conocido, comparando con el valor real $F_2$.


#### 5. HyperLogLog

El **HyperLogLog** es una mejora de LogLog para estimar número de elementos distintos (cardinalidad) con sólo $O(m)$ espacio.

1. **Corrección de sesgo**  
   Explica el uso de la constante $\alpha_m$ y la media armónica en  
   $$
   E = \alpha_m \; m^2 \; \Bigl(\sum_{j=1}^m 2^{-M[j]}\Bigr)^{-1}.
   $$

2. **Errores y tamaño**  
   Muestra que el error relativo estándar es aproximadamente $1.04/\sqrt{m}$.

3. Crea `class HyperLogLog`:

```python
class HyperLogLog:
    def __init__(self, p: int=14, seed: int=None):
        self.m = 1 << p
        self.registers = [0]*self.m
        self.p = p
        self.seed = seed or random.getrandbits(32)
        self.alpha = self._compute_alpha(p)
    def _compute_alpha(self, p):
        if p == 14: return 0.673
        # valores predefinidos
    def add(self, value):
        x = murmurhash3_32(value, self.seed)
        idx = x >> (32-self.p)
        w = (x << self.p) & 0xFFFFFFFF
        rho = self._rank(w, 32-self.p)
        self.registers[idx] = max(self.registers[idx], rho)
    def _rank(self, x, bits):
        return (x & -x).bit_length()  # posición del primer 1
    def count(self):
        Z = 1.0 / sum(2.0**(-r) for r in self.registers)
        E = self.alpha * self.m * self.m * Z
        # correcciones para valores bajos/altos
        return E
```

**Prueba**: inserta $10^6$ claves únicas y mide estimación, compara con verdadero.


In [ ]:
### Tus respuestas